In [7]:
import tensorflow as tf
from keras.preprocessing import image
import numpy as np
import keras
from keras import Model
import time
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import Conv3D

fileName = "Trump.jpg"



# Global MobileNets model
mobile = keras.applications.mobilenet.MobileNet(weights="imagenet")
model = Model(mobile.input, mobile.layers[18].output)


In [ ]:
def write_to_file_input(fileName, X):
    fhandler = open(fileName, "w")
    for i in range(3):
        for j in range(224):
            for k in range(224):
                fhandler.write(str(X[j][k][i]) + "\n")
    fhandler.close()

In [ ]:
def create_Input_Files():
    
    img_width = 224
    img_height = 224

    img = image.load_img(fileName, target_size=(img_width, img_height))
    X = image.img_to_array(img)

    # Normalizing the Input
    X = tf.keras.utils.normalize(X, axis=2)
    
    # Write Normalized Input to file
    write_to_file_input("data/FirstLayer/inputNorm.txt", X)
    
    
    # Write combination input to file
    X[:, :, 0] = 1
    X[:, :, 1] = 10
    X[:, :, 2] = 100
    write_to_file_input("data/FirstLayer/inputComb.txt", X)
    
    # Write all 1's input to file
    X[:, :, 0] = 1
    X[:, :, 1] = 1
    X[:, :, 2] = 1
    write_to_file_input("data/FirstLayer/inputSet1.txt", X)

    print("Creating Input files Complete")


In [ ]:
create_Input_Files()

In [ ]:
def write_to_file_weights_layer_one(fileName, W):
    fTemp = open(fileName, "w")
    for i in range(32):
        for j in range(3):
            for k in range(3):
                for l in range(3):
                    fTemp.write(str(W[k][l][j][i]) + "\n")
    fTemp.close()
    

In [ ]:
W = model.layers[2].get_weights()[0]
write_to_file_weights_layer_one("data/FirstLayer/weightsNorm.txt", W)

W[:,:,:,:] = 1
write_to_file_weights_layer_one("data/FirstLayer/weightsSet1.txt", W)

In [ ]:
def write_to_file_weights_layer_second(fileName, W):
    fTemp = open(fileName, "w")
    for i in range(1):
        for j in range(32):
            for k in range(3):
                for l in range(3):
                    fTemp.write(str(W[k][l][j][i]) + "\n")
    fTemp.close()


In [ ]:
W = model.layers[5].get_weights()[0]
write_to_file_weights_layer_second("data/SecondLayer/weightsNorm.txt", W)

W[:,:,:,:] = 1
write_to_file_weights_layer_second("data/SecondLayer/weightsSet1.txt", W)

In [ ]:
def checkCalculations():

    img_width = 224
    img_height = 224

    img = image.load_img(fileName, target_size=(img_width, img_height))
    X = image.img_to_array(img)

    # Normalized the Input
    X = tf.keras.utils.normalize(X, axis=2)
    
    # Set as 1
    #X[:,:,0] = 1
    #X[:,:,1] = 1
    #X[:,:,2] = 1

    # Set as Combination
    #X[:,:,0] = 1
    #X[:,:,1] = 10
    #X[:,:,2] = 100

    print(X[0][0][0])
    print(X[0][1][0])
    print(X[0][2][0])
    X = np.expand_dims(X, axis=0)
    print(X.shape)

    # Weights part
    W = model.layers[2].get_weights()[0]
    # Weights set as 1
    #W[:,:,:,:] = 1
    W = np.expand_dims(W, axis=0)
    model.layers[2].set_weights(W)
    
    
    W = model.layers[5].get_weights()[0]
    # Weights set as 1
    #W[:,:,:,:] = 1
    W = np.expand_dims(W, axis=0)
    model.layers[5].set_weights(W)
    
    W = model.layers[8].get_weights()[0]
    # Weights set as 1
    #W[:,:,:,:] = 1
    W = np.expand_dims(W, axis=0)
    model.layers[8].set_weights(W)
    
    W = model.layers[12].get_weights()[0]
    # Weights set as 1
    #W[:,:,:,:] = 1
    W = np.expand_dims(W, axis=0)
    model.layers[12].set_weights(W)
    
    W = model.layers[15].get_weights()[0]
    # Weights set as 1
    #W[:,:,:,:] = 1
    W = np.expand_dims(W, axis=0)
    model.layers[15].set_weights(W)
    
    
    output = model.predict(X)
    print("Output Shape ---> ")
    print(output.shape)
    print("--------------------------------")

    # Save Output to file
    fOutput = open("data/FifthLayer/Fifth_Layer_Output.txt", "w")
    for i in range(len(output)):
        for j in range(len(output[0][0][0])):
            for k in range(len(output[0])):
                for l in range(len(output[0][0])):
                    fOutput.write(str(output[i][k][l][j]) + "\n")
    fOutput.close()
    print("Output File Writing Complete!!!")

    print("\n\n Description of Layers")
    for layer in model.layers:
        print(layer)


In [ ]:
import time
start = time.time()
checkCalculations()
end = time.time()
print("Total Time -->", (end - start))


<h2> Checking calculations for Layer 5 </h2>

In [ ]:
def checkFile_FifthLayer():
    fExp = open('data/FifthLayer/Fifth_Layer_Output.txt', 'r')
    fAct = open('data/FifthLayer/output.txt', 'r')
    
    cExp = fExp.readlines()
    cAct = fAct.readlines()
    counter = 0
    for i in range(len(cExp)):
         if int(round(float(cAct[i].strip()))) != int(round(float(cExp[i].strip()))):
            counter += 1
            print(str(cAct[i].strip()) + "  --->   " + str(cExp[i].strip()) )
        
    print("Number of mismatch - " + str(counter))
    
    fExp.close()
    fAct.close()
    
        

In [ ]:
checkFile_FifthLayer()

<h2> Checking calculations for Layer 1 </h2>

In [3]:
def checkFile_FirstLayer():
    fExp = open('data/FirstLayer/First_Layer_Output.txt', 'r')
    fAct = open('data/FirstLayer/output.txt', 'r')
    
    cExp = fExp.readlines()
    cAct = fAct.readlines()
    counter = 0
    for i in range(len(cExp)):
         if int(round(float(cAct[i].strip()))) != int(round(float(cExp[i].strip()))):
            counter += 1
        
    print("Number of mismatch - " + str(counter))
    
    fExp.close()
    fAct.close()
    
        

In [4]:
checkFile_FirstLayer()

Number of mismatch - 388864


<h2> Checking calculations for Layer 4 </h2>

In [ ]:
def checkFile_FourthLayer():
    fExp = open('data/FourthLayer/Fourth_Layer_Output.txt', 'r')
    fAct = open('data/FourthLayer/output.txt', 'r')
    
    cExp = fExp.readlines()
    cAct = fAct.readlines()
    counter = 0
    for i in range(len(cExp)):
         if int(round(float(cAct[i].strip()))) != int(round(float(cExp[i].strip()))):
            counter += 1
            print(str(cAct[i].strip()) + "--->" + str(cExp[i].strip()) )
        
    print("Number of mismatch - " + str(counter))
    
    fExp.close()
    fAct.close()
    
        

In [ ]:
checkFile_FourthLayer()

<h2> Checking calculations for Layer 2 </h2>

In [ ]:
def checkFile_SecondLayer():
    fExp = open('data/SecondLayer/Second_Layer_Output.txt', 'r')
    fAct = open('data/SecondLayer/output.txt', 'r')
    
    cExp = fExp.readlines()
    cAct = fAct.readlines()
    counter = 0
    for i in range(len(cExp)):
         if int(round(float(cAct[i].strip()))) != int(round(float(cExp[i].strip()))):
            counter += 1
            print(str(cAct[i].strip()) + "--->" + str(cExp[i].strip()) )
        
    print("Number of mismatch - " + str(counter))
    
    fExp.close()
    fAct.close()
    
        

In [ ]:
checkFile_SecondLayer()

In [ ]:
def checkFile_ThirdLayer():
    fExp = open('data/ThirdLayer/Third_Layer_Output.txt', 'r')
    fAct = open('data/ThirdLayer/output.txt', 'r')
    
    cExp = fExp.readlines()
    cAct = fAct.readlines()
    counter = 0
    for i in range(len(cExp)):
         if int(round(float(cAct[i].strip()))) != int(round(float(cExp[i].strip()))):
            counter += 1
        
    print("Number of mismatch - " + str(counter))
    
    fExp.close()
    fAct.close()
    
        

<h2> Saving Batch Norm for Layer 2 </h2>

In [ ]:
import math as m
batchNorm = model.layers[6].get_weights()

fMean = open("data/SecondLayer/Second_Layer_Mean.txt", "w")
fSD = open("data/SecondLayer/Second_Layer_StanDev.txt", "w")
fGamma = open("data/SecondLayer/Second_Layer_Gamma.txt", "w")
fBeta = open("data/SecondLayer/Second_Layer_Beta.txt", "w")


for i in range(len(batchNorm[0])):
    fGamma.write(str(float(batchNorm[0][i])) + "\n")

for i in range(len(batchNorm[1])):
    fBeta.write(str(float(batchNorm[1][i])) + "\n")

for i in range(len(batchNorm[2])):
    fMean.write(str(float(batchNorm[2][i])) + "\n")

for i in range(len(batchNorm[3])):
    # The Value for Epsilon is assumed to be 0.001 in this implementation
    fSD.write(str(m.sqrt(float(batchNorm[3][i]) + 0.001)) + "\n")


print(batchNorm[0])
print(batchNorm[1])
print(batchNorm[2])
print(batchNorm[3])

fMean.close()
fSD.close()
fGamma.close()
fBeta.close()


<h2> Saving Weights for Layer 3  </h2>

In [ ]:
def write_to_file_weights_layer_three(fileName, W):
    fTemp = open(fileName, "w")
    for i in range(1):
        for j in range(1):
            for k in range(64):
                for l in range(32):
                    fTemp.write(str(W[i][j][l][k]) + "\n")
    fTemp.close()


In [ ]:
for layer in model.layers:
    print(layer)
    
layer8 = model.layers[8]
W = np.array(layer8.get_weights())
print(W.shape)

W = layer8.get_weights()[0]
write_to_file_weights_layer_three("data/ThirdLayer/weightsNorm.txt", W)

W[:,:,:,:] = 1
write_to_file_weights_layer_three("data/ThirdLayer/weightsSet1.txt", W)

<h2> Saving Batch Norm for Layer 3 </h2>

In [ ]:
import math as m
batchNorm = model.layers[9].get_weights()

fMean = open("data/ThirdLayer/Third_Layer_Mean.txt", "w")
fSD = open("data/ThirdLayer/Third_Layer_StanDev.txt", "w")
fGamma = open("data/ThirdLayer/Third_Layer_Gamma.txt", "w")
fBeta = open("data/ThirdLayer/Third_Layer_Beta.txt", "w")


for i in range(len(batchNorm[0])):
    fGamma.write(str(float(batchNorm[0][i])) + "\n")

for i in range(len(batchNorm[1])):
    fBeta.write(str(float(batchNorm[1][i])) + "\n")

for i in range(len(batchNorm[2])):
    fMean.write(str(float(batchNorm[2][i])) + "\n")

for i in range(len(batchNorm[3])):
    # The Value for Epsilon is assumed to be 0.001 in this implementation
    fSD.write(str(m.sqrt(float(batchNorm[3][i]) + 0.001)) + "\n")


print(batchNorm[0])
print(batchNorm[1])
print(batchNorm[2])
print(batchNorm[3])

fMean.close()
fSD.close()
fGamma.close()
fBeta.close()


<h2> Saving Weights for Layer 4  </h2>

In [ ]:
def write_to_file_weights_layer_four(fileName, W):
    fTemp = open(fileName, "w")
    for i in range(1):
        for j in range(64):
            for k in range(3):
                for l in range(3):
                    fTemp.write(str(W[k][l][j][i]) + "\n")
    fTemp.close()


In [ ]:
for layer in model.layers:
    print(layer)
    
layer12 = model.layers[12]
W = np.array(layer12.get_weights())
print(W.shape)

W = layer12.get_weights()[0]
write_to_file_weights_layer_four("data/FourthLayer/weightsNorm.txt", W)

W[:,:,:,:] = 1
write_to_file_weights_layer_four("data/FourthLayer/weightsSet1.txt", W)

<h2> Saving Batch Norm for Layer 5 </h2>

In [ ]:
import math as m
layer16 = model.layers[16]
W = np.array(layer16.get_weights())
print(W.shape)

batchNorm = model.layers[16].get_weights()

fMean = open("data/FifthLayer/Fifth_Layer_Mean.txt", "w")
fSD = open("data/FifthLayer/Fifth_Layer_StanDev.txt", "w")
fGamma = open("data/FifthLayer/Fifth_Layer_Gamma.txt", "w")
fBeta = open("data/FifthLayer/Fifth_Layer_Beta.txt", "w")


for i in range(len(batchNorm[0])):
    fGamma.write(str(float(batchNorm[0][i])) + "\n")

for i in range(len(batchNorm[1])):
    fBeta.write(str(float(batchNorm[1][i])) + "\n")

for i in range(len(batchNorm[2])):
    fMean.write(str(float(batchNorm[2][i])) + "\n")

for i in range(len(batchNorm[3])):
    # The Value for Epsilon is assumed to be 0.001 in this implementation
    fSD.write(str(m.sqrt(float(batchNorm[3][i]) + 0.001)) + "\n")


print(batchNorm[0])
print(batchNorm[1])
print(batchNorm[2])
print(batchNorm[3])

fMean.close()
fSD.close()
fGamma.close()
fBeta.close()


<h2> Saving Batch Norm for Layer 4 </h2>

In [ ]:
import math as m
layer13 = model.layers[13]
W = np.array(layer13.get_weights())
print(W.shape)

fMean = open("data/FourthLayer/Fourth_Layer_Mean.txt", "w")
fSD = open("data/FourthLayer/Fourth_Layer_StanDev.txt", "w")
fGamma = open("data/FourthLayer/Fourth_Layer_Gamma.txt", "w")
fBeta = open("data/FourthLayer/Fourth_Layer_Beta.txt", "w")


for i in range(len(batchNorm[0])):
    fGamma.write(str(float(batchNorm[0][i])) + "\n")

for i in range(len(batchNorm[1])):
    fBeta.write(str(float(batchNorm[1][i])) + "\n")

for i in range(len(batchNorm[2])):
    fMean.write(str(float(batchNorm[2][i])) + "\n")

for i in range(len(batchNorm[3])):
    # The Value for Epsilon is assumed to be 0.001 in this implementation
    fSD.write(str(m.sqrt(float(batchNorm[3][i]) + 0.001)) + "\n")


print(batchNorm[0])
print(batchNorm[1])
print(batchNorm[2])
print(batchNorm[3])

fMean.close()
fSD.close()
fGamma.close()
fBeta.close()


<h2> Saving Weights for Layer 5  </h2>

In [ ]:
def write_to_file_weights_layer_five(fileName, W):
    fTemp = open(fileName, "w")
    for i in range(1):
        for j in range(128):
            for k in range(1):
                for l in range(64):
                    fTemp.write(str(W[i][k][l][j]) + "\n")
    fTemp.close()


In [ ]:
for layer in model.layers:
    print(layer)
    
layer15 = model.layers[15]
W = np.array(layer15.get_weights())
print(W.shape)

W = layer15.get_weights()[0]
write_to_file_weights_layer_five("data/FifthLayer/weightsNorm.txt", W)

W[:,:,:,:] = 1
write_to_file_weights_layer_five("data/FifthLayer/weightsSet1.txt", W)

<h2> Saving Weights for Layer 6  </h2>

In [9]:
def write_to_file_weights_layer_six(fileName, W):
    fTemp = open(fileName, "w")
    for i in range(1):
        for j in range(128):
            for k in range(3):
                for l in range(3):
                    fTemp.write(str(W[i][k][l][j]) + "\n")
    fTemp.close()


In [10]:
for layer in model.layers:
    print(layer)
    
layer18 = model.layers[18]
W = np.array(layer18.get_weights())
print(W.shape)

W = layer18.get_weights()[0]
write_to_file_weights_layer_six("data/SixthLayer/weightsNorm.txt", W)

W[:,:,:,:] = 1
write_to_file_weights_layer_six("data/SixthLayer/weightsSet1.txt", W)

(1, 3, 3, 128, 1)


FileNotFoundError: [Errno 2] No such file or directory: 'data/SixthLayer/weightsNorm.txt'

<h2> Layer Descriptions </h2>

In [8]:
for layer in model.layers:
    print(layer)
    
layer18 = model.layers[18]
W = np.array(layer18.get_weights())

print("\n\n Weight Size")
print(W.shape)



 Weight Size
(1, 3, 3, 128, 1)
